In [3]:
import bagpy
from bagpy import bagreader
import matplotlib.pyplot as plt
import pandas as pd


In [5]:
bag_name = "first_fly.bag"

b = bagreader(bag_name)

[INFO]  Successfully created the data folder first_fly.


In [14]:
csv_files = []
for topic in b.topics:
    print(topic)
    #data = b.message_by_topic(topic)
    #csv_files.append(data)
    
#b.message_by_topic("/joy")
print(b.topic_table)
b.message_by_topic("/bebop/odometry")

/bebop/odometry
/duration
/joy
            Topics              Types  Message Count    Frequency
0  /bebop/odometry  nav_msgs/Odometry           6438  7256.581315
1        /duration   std_msgs/Float32              1          NaN
2             /joy    sensor_msgs/Joy            392    11.085631


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3: invalid start byte

In [19]:
import rosbag
bag = rosbag.Bag('first_fly.bag')

for topic, msg, t in bag.read_messages(topics=['/bebop_odometry', '/duration']): print(msg)
bag.close()

data: 43.32600021362305
